## Final Project: Phase 3 - Validation with External Indices
Spring 2024  
Group: Michael Massone and Joseph Nelson Farrell   
DS 5230 Unsupervised Machine Learning  
Professor Steven Morin, PhD  
Due: 04/21/2024  
___

In [114]:
# base
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics.cluster import (adjusted_rand_score, contingency_matrix, 
                                     fowlkes_mallows_score, normalized_mutual_info_score)
from sklearn.metrics import jaccard_score, f1_score

# pathing
from pathlib import Path
import os
import sys

### Set Paths

In [115]:
# define path
HOME = Path(os.getcwd())
print(HOME)

HOME_PARENT_STR = str(HOME.parent)
print(HOME_PARENT_STR)

# path to figs folder
PATH_TO_FIGS_FOLDER = HOME_PARENT_STR + '/figs'

# path to data
PATH_TO_DATA_FOLDER = HOME_PARENT_STR + '/data'

# path to src folder
PATH_TO_SRC = HOME_PARENT_STR + '/src'
print(PATH_TO_SRC)

# sys path
sys.path.append(PATH_TO_SRC)

/Users/mikey/LocalFiles/DS5230/final_project/DS5230-final/notebooks
/Users/mikey/LocalFiles/DS5230/final_project/DS5230-final
/Users/mikey/LocalFiles/DS5230/final_project/DS5230-final/src


### Import Functions

In [116]:
import cluster_utils as cu
import external_indices_utils as exi

### Import Results DF & Target DF

In [117]:
# transformed data csv file name
results_file = "/results/all_results_2024-04-1711:49:25.841589.csv"
results_df = pd.read_csv(PATH_TO_DATA_FOLDER + results_file)

target_sample = "/curated/sampled_target.csv"
target_df = pd.read_csv(PATH_TO_DATA_FOLDER + target_sample)

In [118]:
# convert cluster_labels elements back to numpy array
results_df['cluster_labels'] = results_df['cluster_labels'].apply(exi.convert_string_to_array)

In [119]:
#results_df = results_df[results_df['n_clusters_found'] == 7]

In [120]:
target_df

,ID,Target
0,4148,2
1,13224,3
2,9754,6
3,9159,6
4,10139,3
...,...,...
995,13402,3
996,12266,3
997,1657,5
998,3494,1


In [121]:
# get the true number of clusters
num_true_labels = len(np.unique(target_df.Target.values))

___

### Compute External Indices
___

This cell will compute the external indices used to validate the clutering solution.

The following external indices will the used:

* ```Adjusted Rand Score``` <a href="#ref1">[1]</a>
  * The Rand index is a measure similarity between two clustering solutions by considering all pairs of samples and counting pairs that are assigned in the same or different clusters in the predicted and true clusterings.
  * This Rand index is then adjusted for change with:
    <br><br>
    * $ARI = \dfrac{RI - \text{Expected\_RI}}{\textit{max(RI)} - \text{Expected\_RI}}$
* ```Jaccard``` <a href="#ref1">[2]</a>
  * The size of the intersection divided by the size of the union of the two labeled sets.
  * We will use ```average = macro```, this returns the mean of all the independently computed Jaccard indices for each true label. This is acceptble when the true label frequencies are similar.
  <br><br>
* ```Fawlks and Mallows Index```
  * The geometric mean between precision and recall
  <br><br>
  * $FMI = \dfrac{TP}{\sqrt{(TP + FP) \cdot (TP + FN)}}$<a href="#ref1">[3]</a>
  <br><br>
* ```NMI Measure```
* ```F-Measure``` <a href="#ref1">[4]</a>
  * The harmonic mean of the precision and recall.
  <br><br>
  * $F1 = \dfrac{2 \cdot TP}{2 \cdot TP + FP + FN}$
* ```Purity```
  * In this analysis we will compute purity as the proportuon of true labels that were labeled correctly, i.e., that after the best mapping had the same cluster label. We will compute this metric for each true label class and over the entire dataset.

<p id="ref1"><sup>[1]</sup> scikit-learn.org. More details available at <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html">this link</a>.</p>
<p id="ref1"><sup>[2]</sup> scikit-learn.org. More details available at <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.jaccard_score.html">this link</a>.</p>
<p id="ref1"><sup>[3]</sup> scikit-learn.org. More details available at <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.fowlkes_mallows_score.html">this link</a>.</p>
<p id="ref1"><sup>[4]</sup> scikit-learn.org. More details available at <a href="hhttps://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html">this link</a>.</p>



In [122]:
dataframe_dict = {}
df_row_dict_list = []
i = 1
for idx, row in results_df.iterrows():
    print('\n')
    print('*' * 100)
    print(f'Results: {i}')
    print('*'*100)
    print()

    print(f'UMAP & CLUSTER ALGORITHM INFORMATION:')
    print('-'*100)
    
    # get algo
    algo = row['algo']

    ###################################################
    ### Get UMAP & Cluster Algorithm Information ####
    ###################################################

    n_components = row['umap_n_components']
    min_dist = row['umap_min_dist']
    n_neighbors = row['umap_n_neighbors']
    trustworthiness = row['trustworthiness']
    n_clusters_found = row['n_clusters_found']
    validity_index = row['validity_index']
    silhouette_score = row['silhouette_score']
    umap_metric = row['umap_metric']

    print(f"Algorithm: {algo}")
    print(f'Number of Clusers Found: {n_clusters_found}')
    if algo == 'dbscan':
        print(f"Validity Index: {validity_index:.5f}")
    else:
        print(f'Silhouette Score: {silhouette_score:.5f}')
    print(f'UMAP Number of Components: {n_components}')
    print(f'UMAP Min Distance: {min_dist}')
    print(f'UMAP Number of Neighbors: {n_neighbors}')
    print(f'UMAP Metric: {umap_metric}')
    print(f'UMAP Trustworthiness: {trustworthiness:.5f}')

    # create dataframe with columns for every value of n_components
    cluster_labels_df = pd.DataFrame()

    # add cluster labels to dataframe from results_df
    cluster_labels_df.loc[:, str(n_components)] = row['cluster_labels']

    # concatonat target vector dataframe with clusterlabels_df
    labels_df = pd.concat([target_df, cluster_labels_df], axis=1)
    
    # drop indicies with noise points
    noise_points_row_indices = labels_df.index[(labels_df == -1).any(axis=1)]
    labels_df = labels_df.drop(noise_points_row_indices)
    labels_df = labels_df.astype('int64')

    dataframe_dict[f'df_{str(n_components)}'] = labels_df

    # get labels
    true_labels = labels_df.loc[:, 'Target']
    cluster_labels = labels_df.loc[:, str(n_components)]

    ########################################
    ######## Get External Indices #########
    ########################################

    # get adj rand score and add to dataframe
    adj_rand = adjusted_rand_score(true_labels, cluster_labels)
    results_df.loc[idx, 'adjusted_rand_score'] = adj_rand

    # get falks and mallows score
    fawlks_and_mallows_ = fowlkes_mallows_score(true_labels, cluster_labels)

    # get F-1 score
    label = np.unique(true_labels)
    f1_score_ = f1_score(true_labels, cluster_labels, average = 'macro')

    # jaccard score
    jaccard_score_ = jaccard_score(true_labels, cluster_labels, average = 'macro')

    # ge the normalized mutual info score
    nmi = normalized_mutual_info_score(true_labels, cluster_labels)

    # get contigency matrices for all permutatons of cluster labels
    cont_matrix = contingency_matrix(true_labels, cluster_labels)
    matrix_trace = np.trace(cont_matrix)

    # get optimized contingency matrix
    modes_df = exi.get_modes(n_components, labels_df)
    cluster_mapping = exi.get_mapping(n_components, labels_df, modes_df)
    labels_df[str(n_components)] = labels_df.loc[:, str(n_components)].map(cluster_mapping)
    remapped_cont_matrix = contingency_matrix(labels_df['Target'], labels_df[str(n_components)])
    remapped_matrix_trace = np.trace(remapped_cont_matrix)

    # get purities
    purity_df, overall_purity = exi.get_true_label_purity(remapped_cont_matrix, true_labels)

    # retrieve contigency matrix with highest trace
    np.set_printoptions(linewidth=200)
    print()
    print('-'*100)
    print('CONTINGENCY MATRIX')
    print('-'*100)
    #print('Matrix Trace: ', matrix_trace)
    print('Contingency Matrix: ')
    print()
    print(cont_matrix)
    print()
    print('-'*100)
    print('REMAPPED CONTINGENCY MATRIX')
    print('-'*100)
    print()
    print('Best Mapping: ', dict(sorted(cluster_mapping.items())))
    print()
    #print('Remapped Matrix Trace: ', remapped_matrix_trace)
    print('Contingency Matrix: ')
    print()
    print(remapped_cont_matrix)
    print()
    print('-'*100)
    print('EXTERMAL INDICES: PERFORMANCE EVALUATION METRICS')
    print('-'*100)
    print()
    print(f'Adjusted Rand Score: {adj_rand:.5f}')
    print(f'Fawlks and Mallows Score: {fawlks_and_mallows_:.5f}')
    print(f'F-1 Score: {f1_score_:.5f}')
    print(f'Jaccard Score: {jaccard_score_:.5f}')
    print(f'Normalized Mutual Info Score: {nmi:.5f}')
    print()
    print("True Label Purities:")
    print()
    display(purity_df)
    print()
    print(f'Overall Purity: {overall_purity:.5f}')

    df_row_dict_list.append({
                        'algo': algo,
                        'umap_n_components': n_components,
                        'umap_min_dist': min_dist,
                        'umap_n_neighbors': n_neighbors,
                        'umap_metric': umap_metric,
                        'trustworthiness': trustworthiness,
                        'n_clusters_found': n_clusters_found,
                        'true_num_clusters': num_true_labels,
                        'validity_index': validity_index,
                        'adj_rand_score': adj_rand,
                        'fawlks_and_mallows': fawlks_and_mallows_,
                        'nmi': nmi,
                        'jaccard_score': jaccard_score_,
                        'f1_score': f1_score_,
                        'Overall Purity': overall_purity,
                        'true_labels': true_labels,
                        'cluster_labels': cluster_labels,
                        'matrix_trace': matrix_trace,
                        'contingency_matrix': cont_matrix,
                        'remapped_cont_matrix': remapped_cont_matrix,
                        'mapping': cluster_mapping
                        })
    i += 1




****************************************************************************************************
Results: 1
****************************************************************************************************

UMAP & CLUSTER ALGORITHM INFORMATION:
----------------------------------------------------------------------------------------------------
Algorithm: dbscan
Number of Clusers Found: 2
Validity Index: 0.99413
UMAP Number of Components: 2
UMAP Min Distance: 0.0
UMAP Number of Neighbors: 10
UMAP Metric: euclidean
UMAP Trustworthiness: 0.97527

----------------------------------------------------------------------------------------------------
CONTINGENCY MATRIX
----------------------------------------------------------------------------------------------------
Contingency Matrix: 

[[101   0]
 [  0  40]
 [140   0]
 [255   0]
 [129   0]
 [143   0]
 [192   0]]

----------------------------------------------------------------------------------------------------
REMAPPED CONTINGEN

,purity
true_label,
0,0.0
1,1.0
2,0.0
3,1.0
4,0.0
5,0.0
6,0.0



Overall Purity: 0.29500


****************************************************************************************************
Results: 2
****************************************************************************************************

UMAP & CLUSTER ALGORITHM INFORMATION:
----------------------------------------------------------------------------------------------------
Algorithm: dbscan
Number of Clusers Found: 59
Validity Index: 0.55032
UMAP Number of Components: 2
UMAP Min Distance: 0.0
UMAP Number of Neighbors: 10
UMAP Metric: canberra
UMAP Trustworthiness: 0.94979

----------------------------------------------------------------------------------------------------
CONTINGENCY MATRIX
----------------------------------------------------------------------------------------------------
Contingency Matrix: 

[[ 2  1  0  0  0  3  0 11  1  5 14  0  0  0  0  0  0  0  0  0  0  0  0  0  1 18  0  0  0  0  0  0  2  0  0  0  0  7  0  0  1  0  7  0  8  0  5  0  1  0  0  0  0  0  0  0  0 

,purity
true_label,
0,0.057471
1,0.350000
2,0.034783
3,0.013953
4,0.040650
5,0.046154
6,0.027586



Overall Purity: 0.90292


****************************************************************************************************
Results: 3
****************************************************************************************************

UMAP & CLUSTER ALGORITHM INFORMATION:
----------------------------------------------------------------------------------------------------
Algorithm: dbscan
Number of Clusers Found: 2
Validity Index: 0.99716
UMAP Number of Components: 2
UMAP Min Distance: 0.0
UMAP Number of Neighbors: 10
UMAP Metric: chebyshev
UMAP Trustworthiness: 0.97324

----------------------------------------------------------------------------------------------------
CONTINGENCY MATRIX
----------------------------------------------------------------------------------------------------
Contingency Matrix: 

[[101   0]
 [  0  40]
 [140   0]
 [255   0]
 [129   0]
 [143   0]
 [192   0]]

----------------------------------------------------------------------------------------------

...


Overall Purity: 0.93419


****************************************************************************************************
Results: 191
****************************************************************************************************

UMAP & CLUSTER ALGORITHM INFORMATION:
----------------------------------------------------------------------------------------------------
Algorithm: dbscan
Number of Clusers Found: 2
Validity Index: 0.26712
UMAP Number of Components: 5
UMAP Min Distance: 0.5
UMAP Number of Neighbors: 200
UMAP Metric: chebyshev
UMAP Trustworthiness: 0.92098

----------------------------------------------------------------------------------------------------
CONTINGENCY MATRIX
----------------------------------------------------------------------------------------------------
Contingency Matrix: 

[[101   0]
 [  0  40]
 [140   0]
 [255   0]
 [129   0]
 [143   0]
 [192   0]]

-------------------------------------------------------------------------------------------

,purity
true_label,
0,0.0
1,1.0
2,0.0
3,1.0
4,0.0
5,0.0
6,0.0



Overall Purity: 0.29500


****************************************************************************************************
Results: 192
****************************************************************************************************

UMAP & CLUSTER ALGORITHM INFORMATION:
----------------------------------------------------------------------------------------------------
Algorithm: k_means
Number of Clusers Found: 5
Silhouette Score: 0.58925
UMAP Number of Components: 5
UMAP Min Distance: 0.5
UMAP Number of Neighbors: 200
UMAP Metric: correlation
UMAP Trustworthiness: 0.94043

----------------------------------------------------------------------------------------------------
CONTINGENCY MATRIX
----------------------------------------------------------------------------------------------------
Contingency Matrix: 

[[100   1]
 [ 40   0]
 [140   0]
 [  2 253]
 [124   5]
 [  1 142]
 [ 32 160]]

--------------------------------------------------------------------------------------

,purity
true_label,
0,0.000000
1,0.000000
2,1.000000
3,0.992157
4,0.000000
5,0.000000
6,0.000000



Overall Purity: 0.39300


In [126]:
matrix_results_df = pd.DataFrame(df_row_dict_list)

matrix_results_df.sort_values(by='adj_rand_score', ascending=False).head(10)

,algo,umap_n_components,umap_min_dist,umap_n_neighbors,umap_metric,trustworthiness,n_clusters_found,true_num_clusters,validity_index,adj_rand_score,...,nmi,jaccard_score,f1_score,Overall Purity,true_labels,cluster_labels,matrix_trace,contingency_matrix,remapped_cont_matrix,mapping
27,dbscan,2,0.1,100,correlation,0.922158,6,7,0.472575,0.614946,...,0.736064,0.007115,0.013556,0.757545,0 2 1 3 2 6 3 6 4 3 ...,0 0 1 1 2 1 3 1 4 1 ...,13,"[[13, 2, 0, 86, 0, 0], [0, 0, 0, 0, 40, 0], [1...","[[86, 0, 13, 2, 0, 0], [0, 40, 0, 0, 0, 0], [0...","{2.0: 5.0, 5.0: 4.0, 1.0: 3.0, 0.0: 2.0, 4.0: ..."
7,dbscan,2,0.0,25,correlation,0.949226,6,7,0.475030,0.611930,...,0.736231,0.005559,0.010701,0.756784,0 2 1 3 2 6 3 6 4 3 ...,0 0 1 1 2 1 3 1 4 1 ...,10,"[[10, 2, 0, 86, 0, 0], [0, 0, 0, 0, 40, 0], [1...","[[86, 0, 10, 2, 0, 0], [0, 40, 0, 0, 0, 0], [0...","{2.0: 5.0, 5.0: 4.0, 1.0: 3.0, 0.0: 2.0, 4.0: ..."
107,dbscan,4,0.0,100,correlation,0.929327,6,7,0.746248,0.611901,...,0.731254,0.005559,0.010701,0.756811,0 2 1 3 2 6 3 6 4 3 ...,0 0 1 1 2 1 3 1 4 1 ...,10,"[[10, 2, 0, 86, 0, 0], [0, 0, 0, 0, 40, 0], [1...","[[86, 0, 10, 2, 0, 0], [0, 40, 0, 0, 0, 0], [0...","{2.0: 5.0, 5.0: 4.0, 1.0: 3.0, 0.0: 2.0, 4.0: ..."
111,dbscan,4,0.0,200,correlation,0.931698,6,7,0.782335,0.609613,...,0.729120,0.006138,0.011771,0.755040,0 2 1 3 2 6 3 6 4 3 ...,0 0 1 1 2 1 3 1 4 1 ...,11,"[[11, 2, 0, 86, 0, 0], [0, 0, 0, 0, 40, 0], [1...","[[86, 0, 11, 2, 0, 0], [0, 40, 0, 0, 0, 0], [0...","{2.0: 5.0, 5.0: 4.0, 1.0: 3.0, 0.0: 2.0, 4.0: ..."
155,dbscan,5,0.0,100,correlation,0.928126,6,7,0.720153,0.609241,...,0.728361,0.006138,0.011771,0.755040,0 2 1 3 2 6 3 6 4 3 ...,0 0 1 1 2 1 3 1 4 1 ...,11,"[[11, 2, 0, 86, 0, 0], [0, 0, 0, 0, 40, 0], [1...","[[86, 0, 11, 2, 0, 0], [0, 40, 0, 0, 0, 0], [0...","{2.0: 5.0, 5.0: 4.0, 1.0: 3.0, 0.0: 2.0, 4.0: ..."
11,dbscan,2,0.0,100,correlation,0.928587,6,7,0.678127,0.607028,...,0.725856,0.007088,0.013506,0.753000,0 2 1 3 2 6 3 6 4 3 ...,0 0 1 1 2 1 3 1 4 1 ...,13,"[[13, 2, 0, 86, 0, 0], [0, 0, 0, 0, 40, 0], [1...","[[86, 0, 13, 2, 0, 0], [0, 40, 0, 0, 0, 0], [0...","{2.0: 5.0, 5.0: 4.0, 1.0: 3.0, 0.0: 2.0, 4.0: ..."
159,dbscan,5,0.0,200,correlation,0.929874,6,7,0.758307,0.606521,...,0.725430,0.007170,0.013655,0.753012,0 2 1 3 2 6 3 6 4 3 ...,0 0 1 1 2 1 3 1 4 1 ...,13,"[[13, 2, 0, 86, 0, 0], [0, 0, 0, 0, 40, 0], [1...","[[86, 0, 13, 2, 0, 0], [0, 40, 0, 0, 0, 0], [0...","{2.0: 5.0, 5.0: 4.0, 1.0: 3.0, 0.0: 2.0, 4.0: ..."
106,dbscan,4,0.0,100,chebyshev,0.934863,7,7,0.563657,0.595640,...,0.735231,0.005421,0.010522,0.754497,0 2 2 6 3 6 5 5 6 3 ...,0 0 2 1 3 1 5 2 6 1 ...,10,"[[9, 2, 0, 85, 0, 0, 0], [0, 0, 0, 0, 40, 0, 0...","[[85, 0, 9, 2, 0, 0, 0], [0, 40, 0, 0, 0, 0, 0...","{2.0: 5.0, 6.0: 4.0, 5.0: 8, 1.0: 3.0, 0.0: 2...."
99,dbscan,4,0.0,10,correlation,0.956564,6,7,0.694637,0.594545,...,0.709374,0.005371,0.010352,0.751000,0 2 1 3 2 6 3 6 4 3 ...,0 0 1 1 2 1 3 1 4 1 ...,10,"[[10, 2, 0, 89, 0, 0], [0, 0, 0, 0, 40, 0], [1...","[[89, 0, 10, 2, 0, 0], [0, 40, 0, 0, 0, 0], [1...","{2.0: 5.0, 5.0: 4.0, 1.0: 3.0, 0.0: 2.0, 4.0: ..."
51,dbscan,3,0.0,10,correlation,0.957196,6,7,0.585390,0.593121,...,0.706101,0.005371,0.010352,0.750000,0 2 1 3 2 6 3 6 4 3 ...,0 0 1 1 2 1 3 1 4 1 ...,10,"[[10, 2, 0, 89, 0, 0], [0, 0, 0, 0, 40, 0], [1...","[[89, 0, 10, 2, 0, 0], [0, 40, 0, 0, 0, 0], [1...","{2.0: 5.0, 5.0: 4.0, 1.0: 3.0, 0.0: 2.0, 4.0: ..."


In [124]:
keep_cols = ['true_num_clusters', 'umap_n_components','umap_min_dist', 
             'umap_n_neighbors', 'umap_metric', 'trustworthiness', 'algo', 'n_clusters_found',
            'validity_index', 'adj_rand_score', 'fawlks_and_mallows', 'nmi', 'jaccard_score', 'f1_score']

finalized_results_frame = matrix_results_df[keep_cols]

finalized_results_frame

,true_num_clusters,umap_n_components,umap_min_dist,umap_n_neighbors,umap_metric,trustworthiness,algo,n_clusters_found,validity_index,adj_rand_score,fawlks_and_mallows,nmi,jaccard_score,f1_score
0,7,2,0.0,10,euclidean,0.975270,dbscan,2,0.994125,0.033437,0.428567,0.167015,0.157887,0.170055
1,7,2,0.0,10,canberra,0.949785,dbscan,59,0.550322,0.180227,0.325349,0.557934,0.000394,0.000776
2,7,2,0.0,10,chebyshev,0.973240,dbscan,2,0.997160,0.033437,0.428567,0.167015,0.157887,0.170055
3,7,2,0.0,10,correlation,0.956992,dbscan,7,0.568871,0.588492,0.676654,0.699517,0.022513,0.040964
4,7,2,0.0,25,euclidean,0.964574,dbscan,2,0.994259,0.033437,0.428567,0.167015,0.157887,0.170055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,7,5,0.5,100,correlation,0.931711,k_means,4,NaN,0.281836,0.535713,0.442663,0.032468,0.052910
188,7,5,0.5,200,euclidean,0.961939,dbscan,2,0.594161,0.033437,0.428567,0.167015,0.157887,0.170055
189,7,5,0.5,200,canberra,0.911820,dbscan,34,0.170262,0.456118,0.570869,0.662987,0.005474,0.009971
190,7,5,0.5,200,chebyshev,0.920981,dbscan,2,0.267117,0.033437,0.428567,0.167015,0.157887,0.170055
